In [1]:
import pandas as pd
DATADIR = "D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/trainLabels.csv"

trainLabels = pd.read_csv(DATADIR)
trainLabels.head()


,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [3]:
from PIL import Image
from keras.preprocessing import image
import os
import numpy as np

# resize the image to (256, 256)
img_rows, img_cols = 256, 256

listing = os.listdir("D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/resized_train/resized_train/") 
'''listing.remove("trainLabels.csv")'''

immatrix = []
imlabel = []

listing

['10003_left.jpeg',
 '10003_right.jpeg',
 '10007_left.jpeg',
 '10007_right.jpeg',
 '10009_left.jpeg',
 '10009_right.jpeg',
 '1000_left.jpeg',
 '1000_right.jpeg',
 '10010_left.jpeg',
 '10010_right.jpeg',
 '10013_left.jpeg',
 '10013_right.jpeg',
 '10014_left.jpeg',
 '10014_right.jpeg',
 '10015_left.jpeg',
 '10015_right.jpeg',
 '10017_left.jpeg',
 '10017_right.jpeg',
 '10022_left.jpeg',
 '10022_right.jpeg',
 '10028_left.jpeg',
 '10028_right.jpeg',
 '10029_left.jpeg',
 '10029_right.jpeg',
 '1002_left.jpeg',
 '1002_right.jpeg',
 '10030_left.jpeg',
 '10030_right.jpeg',
 '10031_left.jpeg',
 '10031_right.jpeg',
 '10032_left.jpeg',
 '10032_right.jpeg',
 '10035_left.jpeg',
 '10035_right.jpeg',
 '10042_left.jpeg',
 '10042_right.jpeg',
 '10043_left.jpeg',
 '10043_right.jpeg',
 '10046_left.jpeg',
 '10046_right.jpeg',
 '10047_left.jpeg',
 '10047_right.jpeg',
 '10048_left.jpeg',
 '10048_right.jpeg',
 '10050_left.jpeg',
 '10050_right.jpeg',
 '10053_left.jpeg',
 '10053_right.jpeg',
 '10058_left.jpeg',


In [5]:
for file in listing:
    base = os.path.basename("D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/resized_train/resized_train/" + file)
    fileName = os.path.splitext(base)[0]
    imlabel.append(trainLabels.loc[trainLabels.image==fileName, 'level'].values[0])
    im = Image.open("D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/resized_train/resized_train/" + file)
    img = np.array(im.resize((img_rows,img_cols)))
    
    # convert to green channel only
    img[:,:,[0,2]] = 0
    immatrix.append(img)
    length = len(immatrix)
    if(length % 500 == 0):
        print(length)

In [6]:
im = Image.fromarray(immatrix[1],'RGB')
print("level:",imlabel[1])
im

IndexError: list index out of range

In [ ]:
import random

# define transformation methods
def horizontal_flip(image_array):
    return image_array[:, ::-1]

def vertical_flip(image_array):
    return image_array[::-1,:]

def random_transform(image_array):
    if random.random() < 0.5:
        return vertical_flip(image_array)
    else:
        return horizontal_flip(image_array)

In [ ]:
im = Image.fromarray(vertical_flip(immatrix[1]),'RGB')
im

In [ ]:
length = len(immatrix)
for i in range(length):
    if random.random() < 0.1:
        immatrix.append(random_transform(immatrix[i]))
        imlabel.append(imlabel[i])
        
print("Size of image array before augmentation: ", length)
print("Size fo image array after augmentation: ", len(immatrix))

In [ ]:
from sklearn.utils import shuffle

data,label = shuffle(immatrix, imlabel, random_state=42)
train_data = [data,label]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data[0], train_data[1], test_size = 0.1, random_state = 42)

print(np.array(x_train).shape)
print(np.array(y_train).shape)

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(np.array(y_train), 5)
y_test = np_utils.to_categorical(np.array(y_test), 5)

x_train = np.array(x_train).astype("float32")/255.
x_test = np.array(x_test).astype("float32")/255.

print(np.array(y_train).shape)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train[0].shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(activation='softmax', units=5))

model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size = 1, epochs=10, shuffle=True, verbose=2)


In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print(score)